In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
DATASET_PATH = './dataset'

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

train_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_gen = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

print("Class indices:", train_gen.class_indices)


Found 15811 images belonging to 4 classes.
Found 3951 images belonging to 4 classes.
Class indices: {'Harmful': 0, 'Organic': 1, 'Recyclable': 2, 'Residual': 3}


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Build the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(4, activation='softmax')  # 4 output classes
])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Setup early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
history = model.fit(
    train_gen,
    epochs=10,
    validation_data=val_gen,
    callbacks=[early_stop]
)


Epoch 1/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 0s 810ms/step - accuracy: 0.5994 - loss: 0.9137

c:\Users\pairkshith\VS Code projects\PBL\WasteClassifier2.0\waste-env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


495/495 ━━━━━━━━━━━━━━━━━━━━ 497s 1s/step - accuracy: 0.5995 - loss: 0.9135 - val_accuracy: 0.7864 - val_loss: 0.5889
Epoch 2/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 490s 991ms/step - accuracy: 0.6875 - loss: 0.7044 - val_accuracy: 0.7730 - val_loss: 0.5885
Epoch 3/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 214s 433ms/step - accuracy: 0.7206 - loss: 0.6465 - val_accuracy: 0.7712 - val_loss: 0.5711
Epoch 4/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 201s 405ms/step - accuracy: 0.7471 - loss: 0.5976 - val_accuracy: 0.7879 - val_loss: 0.5453
Epoch 5/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 210s 423ms/step - accuracy: 0.7631 - loss: 0.5575 - val_accuracy: 0.7160 - val_loss: 0.5950
Epoch 6/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 208s 421ms/step - accuracy: 0.7796 - loss: 0.5304 - val_accuracy: 0.7952 - val_loss: 0.5127
Epoch 7/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 237s 480ms/step - accuracy: 0.7965 - loss: 0.4948 - val_accuracy: 0.8469 - val_loss: 0.4225
Epoch 8/10
495/495 ━━━━━━━━━━━━━━━━━━━━ 204s 412ms/step - accuracy: 0.8089 - loss: 0.4794 

In [6]:
model.save("waste_classifier.h5")


In [7]:
import json

# Save class indices
with open("labels.json", "w") as f:
    json.dump(train_gen.class_indices, f)


In [2]:
import tensorflow as tf
print(tf.__version__)

2.19.0
